# I'm Something of a Painter Myself
## Akriti Prajapati, Gaurav Aryal, Gaurav Aryal

### The project we worked off of: https://www.kaggle.com/xyzymir/cycle-gan-pytorch

#### Helper Projects: 
    https://www.kaggle.com/atrisaxena/pytorch-cyclegan
    https://www.kaggle.com/nachiket273/cyclegan-pytorch
    https://www.kaggle.com/adrianda/cyclegan-pytorch-style-transfer

Every artist has their own style that they demonstrate in their art pieces. This project is to bring Claude Monte’s style into photos or create the style from scratch. Generative adversarial networks (GANs) is a class of machine learning frameworks which made imitation of those arts possible. In this project, the task is to build GAN that generates 7,000 images in the style of Monet.

### Brief intro on what we changed:
We took the above mentioned project as a base project and started working forward with it. Following are what we have changed or added along the way:

1. Set-up
-> We added necessary imports as we made changes to the project

2. DataSet
-> Instead of using cv2.imread method from OpenCV library, we used "Image" method from PIL as it was easier to handle images through that method.

3. Data Augmentation
-> There are only 300 monet images while there are more than 7000 photos, so we decided to used data augmentation and add in some tweaked images into the monet dataset to increase its count. 

4. Reverse Normalize Method
-> In order to undo the what the transforms.normalize does to our images.

5. Test Images
-> We plotted original photo and monet images to have a peak at what our datasets look like.

6. CycleGAN Models
-> We re-wrote all the helping functinos and discriminator, generator methods. 

7. Loss Functions 
-> Along with adversarial loss function and cycle consistency loss function, we also added in identity loss function to further ensure that the output from a mapping visually match the image from the domain they map to.

8. Training Method
-> The training method was quite well done, we did not change much in there but certainly did calculated all those loss functions.

9. Image and Graph Plot
-> After finishing the training, we plotted some images to compare and also a graph to showcase our training loss values.

10. Submition
-> We made some changes to accomodate our previous changes in PictureDataset.

## Set-up
All the packages are imported here.

In [ ]:
import torch
from torch import tensor
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import numpy as np
import pandas as pd
import random

# for loading images
from pathlib import Path
# import cv2
from torch.utils.data import Dataset, DataLoader
import PIL
from PIL import Image
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms

import matplotlib.pyplot as plt
from torchvision.utils import make_grid


# progress bar
from fastprogress import master_bar, progress_bar

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
path = Path('../input/gan-getting-started')
Path.BASE_PATH = path
Path.ls = lambda x: [o.name for o in x.iterdir()]

# Datasets
PictureDataset is returning a dataset for given path. Here, we have added some data augmentation. We have added some transformations to augment our data. As we already know, there are only 300 monet images which are very less with respect to the dataset of other photos. Inorder to increase the monet dataset, we concatenated augmented images with our original monet images. This method will augment our data whenever augment=True.

Here, we have used the inbuild functionalitites of transtorchvision.transforms such as Resize, CenterCrop, RandomHorizontalFlip and CenterCrop to transform our data.

In [ ]:
class PictureDataset(Dataset):
    def __init__(self, df, folder_path, img_size=256, augment=True):
        super().__init__()
        self.df = df
        self.folder_path = folder_path
        
        # Normalization helps get data within a range and reduces the skewness 
        # which helps learn faster and better
        # Normalize does the following for each channel: image = (image - mean) / std 
        # The parameters mean, std are passed as 0.5 and 0.5 respectively. 
        # This will normalize the image in the range [-1,1]
        if augment:
            self.transform = transforms.Compose([
                transforms.Resize(img_size),
                transforms.CenterCrop(256),
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
            ])
        else:
            self.transform = transforms.Compose([
                transforms.Resize(img_size),
                transforms.ToTensor(),
                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
            ])
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        img = Image.open(str(self.folder_path/self.df[0][idx]))
        img = self.transform(img)
        
        return img


## Reverse Normalization
This unnormalize_img method undo what normalization does.
The zip() function takes in iterables as arguments and returns an iterator. This iterator generates a series of tuples containing elements from each iterable. We iterate through the given mean and standard deviation values to reverse what we did in normalization.

In [ ]:
def unnormalize_img(img, mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]):
    for t, m, s in zip(img, mean, std):
        t.mul_(s).add_(m)
    
    return img

In [ ]:
# create dataframe
df_photo = pd.DataFrame((path/'photo_jpg').ls())
df_monet = pd.DataFrame((path/'monet_jpg').ls())

# photo dataset
photoDS = PictureDataset(df_photo, path/'photo_jpg', img_size=256, augment=False)

# original monet dataset
original_monet = PictureDataset(df_monet, path/'monet_jpg', img_size=256, augment=False)
#transformed monet dataset
augmented_monet = PictureDataset(df_monet, path/'monet_jpg', img_size=300, augment=True)
# monetDS is the dataset which contains both the augmented and original monet images
monetDS = torch.utils.data.ConcatDataset([augmented_monet, original_monet])

batch_size = 100

# create loaders
photo_loader = DataLoader(photoDS, batch_size, shuffle=True, num_workers=0, drop_last=True)
monet_loader = DataLoader(monetDS, batch_size, shuffle=True, num_workers=0, drop_last=True)

## Test Images
We have plotted some photos and monet images.

In [ ]:
# Create a test dataloader and plot the monet and photo images
test_batch_size = 3
test_photo_dataloader = DataLoader(photoDS, test_batch_size, shuffle=True, num_workers=0, drop_last=True)
test_monet_dataloader = DataLoader(monetDS, test_batch_size, shuffle=True, num_workers=0, drop_last=True)

# plot the photo images
img_iter = iter(test_photo_dataloader)
img = img_iter.next()
grid_normalized = make_grid(img, nrow=4)
grid_original = unnormalize_img(grid_normalized)
fig = plt.figure(figsize=(8, 8))
plt.title('photo')
plt.imshow(grid_original.permute(1, 2, 0).detach().numpy())
plt.show()

# plot the monet images
img_iter = iter(test_monet_dataloader)
img = img_iter.next()
grid_normalized = make_grid(img, nrow=4)
grid_original = unnormalize_img(grid_normalized)
fig = plt.figure(figsize=(8, 8))
plt.title('monet')
plt.imshow(grid_original.permute(1, 2, 0).detach().numpy())
plt.show()

## Building blocks
There are three helping blocks.
The ConvBlock returns sequence of convolutional layer followed by batch normalization if batch_norm=True.
We previously chose to place instance normalization along with batch normalization. Batch normalization computes one mean and std dev (thus making the distribution of the whole layer Gaussian), instance normalization computes T (the number of input tensors) of them, making each individual image distribution look Gaussian, but not jointly. Before we used only 1 batch size so it was more benefitial to use instance normalization but now we are using 100 batch sizes, so it made more sense to use batch normalization and it generated better monet picture however the generator loss keeps increasing.

In [ ]:
def ConvBlock(in_channels , out_channels , kernel_size=4 , strides=2, padding=1, batch_norm=True):
    layers = []      
    conv_layer = nn.Conv2d(in_channels=in_channels , out_channels=out_channels , 
                          kernel_size=kernel_size , stride=strides , padding=padding ,  bias=False)
    layers.append(conv_layer)
    if(batch_norm):
        layers.append(nn.BatchNorm2d(out_channels))
        
    return nn.Sequential(*layers)


For DeconvBlock, we initially had just a transpose convolutional layer with optional batch normalization but it started to leave checkboard marks on the generated images. After one of our calssmates presentation where he suspected the reason could be because of deconvolution operation. We did a little more research and found it to be true so then we tried and implemented the nn.Upsample method from the referred project which seem to have solved the checkboard issue.


In [ ]:
class DeconvBlock(nn.Sequential):
    def __init__(self, in_channels, out_channels, kernel_size=3, padding=1, batch_norm=True):
        super().__init__(
            nn.Upsample(scale_factor=2),
            ConvBlock(in_channels, out_channels, kernel_size, strides=1, padding=padding, batch_norm=batch_norm)
        )

The ResidualBlock consists of two convolutional layers followed by batch normalization and relu activation function where a residue of input is added to the output. This is done to ensure properties of input of previous layers are available for later layers as well, so that their output do not deviate much from original input. Otherwise the characteristics of original images will not be retained in the output and results will be very abrupt.

In [ ]:
 class ResidualBlock(nn.Module):
    def __init__(self, in_features):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Conv2d(in_features, in_features, kernel_size=3, padding=1),
            nn.BatchNorm2d(in_features),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_features, in_features, kernel_size=3, padding=1),
            nn.BatchNorm2d(in_features),
            nn.ReLU(inplace=True)
        )
    
    def forward(self, x):
        return x + self.layers(x)


## Generator
The generator consists of three convolutional blocks, followed by a few resnet blocks, then three deconvoltional blocks.

We used leaky relu as the activation function for encoder and decoder and just relu function in residual block. The leaky relu is used to avoid the "dead relu" problem which happens when your ReLU always have values under 0. The residual block connects the output of one layer with the input of an earlier layer, so even with relu, we might lose some characteristics but we won't face dead relu.

In [ ]:
class CGanGenerator(nn.Module):
    def __init__(self, conv_dim=8, res_layers=6):
        super(CGanGenerator, self).__init__()

        #Encoder layers
        self.conv1 = ConvBlock(in_channels=3, out_channels=conv_dim, kernel_size=5, padding=2)
        self.conv2 = ConvBlock(in_channels=conv_dim, out_channels=conv_dim*2, kernel_size=3, batch_norm=True)
        self.conv3 = ConvBlock(in_channels=conv_dim*2, out_channels=conv_dim*4, kernel_size=3, batch_norm=True)
        
        #Residual blocks
        residual_layers = []
        for layer in range(res_layers):
            residual_layers.append(ResidualBlock(conv_dim*4))
        self.res_blocks = nn.Sequential(*residual_layers)
        
        #Decoder layers
        self.deconv4 = DeconvBlock(in_channels=conv_dim*4, out_channels=conv_dim*2, kernel_size=3, batch_norm=True)
        self.deconv5 = DeconvBlock(in_channels=conv_dim*2, out_channels=conv_dim, kernel_size=3, batch_norm=True)
        self.deconv6 = DeconvBlock(in_channels=conv_dim, out_channels=3, kernel_size=3, batch_norm=True)
        
    def forward(self, x):
        #Encoder
        out = F.leaky_relu(self.conv1(x), negative_slope=0.2)
        out = F.leaky_relu(self.conv2(out), negative_slope=0.2)
        out = F.leaky_relu(self.conv3(out), negative_slope=0.2)
        
        #Residual blocks
        out = self.res_blocks(out)
        
        #Decoder
        out = F.leaky_relu(self.deconv4(out), negative_slope=0.2)
        out = F.leaky_relu(self.deconv5(out), negative_slope=0.2)
        out = torch.tanh(self.deconv6(out))
        
        return out

## Discriminator
Each generator has a corresponding discriminator model.
Our discriminator consists of bucnh of convlolutional layers. The convolution network help to extract features from the images which may be helpful in classifying the objects in that image. It starts by extracting low dimensional features (like edges) from the image, and then some high dimensional features like the shapes. We implemented a simplified CycleGAN discriminator, which is a network of 6 convolution layers with optional batch normalization including:
5 layers to extract features from the image, and
1 layer to produce the output (whether the image is fake or not).

In [ ]:
class CGanDiscriminator(nn.Module):
    def __init__(self, conv_dim=8):
        super(CGanDiscriminator, self).__init__()
        
        #convolutional layers, increasing in depth
        self.conv1 = ConvBlock(in_channels=3, out_channels=conv_dim, kernel_size=5)
        self.conv2 = ConvBlock(in_channels=conv_dim, out_channels=conv_dim*2, kernel_size=3, batch_norm=True)
        self.conv3 = ConvBlock(in_channels=conv_dim*2, out_channels=conv_dim*4, kernel_size=3, batch_norm=True)
        self.conv4 = ConvBlock(in_channels=conv_dim*4, out_channels=conv_dim*8, kernel_size=3, batch_norm=True)
        self.conv5 = ConvBlock(in_channels=conv_dim*8, out_channels=conv_dim*8, kernel_size=3, batch_norm=True)
        self.conv6 = ConvBlock(conv_dim*8, out_channels=1, kernel_size=3, strides=2)
    
    def forward(self, x):
        #leaky relu applied to all conv layers but last
        out = F.leaky_relu(self.conv1(x), negative_slope=0.2)
        out = F.leaky_relu(self.conv2(out), negative_slope=0.2)
        out = F.leaky_relu(self.conv3(out), negative_slope=0.2)
        out = F.leaky_relu(self.conv4(out), negative_slope=0.2)
        out = F.leaky_relu(self.conv5(out), negative_slope=0.2)
        out = torch.sigmoid(self.conv6(out))
       
        return out 

### Build model and optimizer
We have build two generators which converts monet into photo and photo into monet respectively. Then we have two generators.

Optimizers help to adjust the properties of a network, such as weights and learning rate to minimize the losses. Adadelta is an Adagrad extension reduces the aggressive decreasing learning rate. Adadelta limits the window of cumulative past gradients to a certain fixed size w, instead of accumulating all previous squared gradients.


In [ ]:
monet_to_photo = CGanGenerator(res_layers=2).to(device)
photo_to_monet = CGanGenerator(res_layers=2).to(device)

monet_dis = CGanDiscriminator().to(device) # checks photo->monet
photo_dis = CGanDiscriminator().to(device) # checks monet->photo

opt_m2p = optim.Adadelta(monet_to_photo.parameters())
opt_p2m = optim.Adadelta(photo_to_monet.parameters())

opt_mdis = optim.Adadelta(monet_dis.parameters())
opt_pdis = optim.Adadelta(photo_dis.parameters())

### Loss Function
The project had defined adversarial loss function and cycle consistenct loss function within the training loop but we wanted to separate them into a new function and add in another loss function: identity loss function.

#### Adversarial Loss Function
Both generators are attempting to “fool” their corresponding discriminator into being less able to distinguish their generated images from the real versions. We used the least squares loss to capture this.

#### Cycle Consistency Loss Function
However, the adversarial loss alone is not sufficient to produce good images, as it leaves the model under-constrained. It enforces that the generated output be of the appropriate domain, but does not enforce that the input and output are recognizably the same. For example, a generator that output an image y that was an excellent example of that domain, but looked nothing like x, would do well by the standard of the adversarial loss, despite not giving us what we really want.

The cycle consistency loss addresses this issue. It relies on the expectation that if you convert an image to the other domain and back again, by successively feeding it through both generators, you should get back something similar to what you put in. It enforces that F(G(x)) ≈ x and G(F(y)) ≈ y.

#### Identity Loss Function
The identity loss function further ensures that the outputs from a mapping visually match the image from the domain they map to.

This loss can regularize the generator to be near an identity mapping when real samples of the target domain are provided. If something already looks like from the target domain, you should not map it into a different image. It  is used to preserve the color and prevent reverse color in the result. We calculate identity loss by doing absolute difference of generated image from real image times some identity weight.

Forward identity loss X->Y. Backward identity loss from Y->X.


In [ ]:
# Adversarial Loss
def real_mse_loss(D_out, adverserial_weight=1):
    return torch.mean((D_out-1)**2) * adverserial_weight

# Adversarial Loss
def fake_mse_loss(D_out, adverserial_weight=1):
    return torch.mean(D_out**2) * adverserial_weight

# Cycle Consistency Loss
def cycle_consistency_loss(real_img, reconstructed_img, lambda_weight=1):
    reconstr_loss = torch.mean(torch.abs(real_img - reconstructed_img))
    return lambda_weight * reconstr_loss 

# Identity Loss
def identity_loss(real_img, generated_img, identity_weight=0.5):
    ident_loss = torch.mean(torch.abs(real_img - generated_img))
    return identity_weight * ident_loss

## Training Loop
The training method ModelTraining was well formed in the referred project, however we added some loss functions and printed the progress.

In [ ]:
def ModelTraining(epochs, discriminator_epochs = 5):
    mb = master_bar(range(epochs))

    losses = [] #losses over all iteration
    
    # additional weighting parameters
    adverserial_weight = 0.5
    lambda_weight = 10
    identity_weight = 5
    
    #Average loss over batches per epoch runs
    d_total_loss_avg = 0.0
    g_total_loss_avg = 0.0
    
    # minimum number of images in the iteration
    monet_iter = iter(monet_loader)
    img_per_iteration = len(monet_iter)
    
    for epoch in mb:
        # do a few discriminator epochs
        
        # declare losses that will be used in reach batch iteration
        # monet discriminator losses
        realMonetLoss = 0.0
        fakeMonetLoss = 0.0
        # photo discriminator losses
        realPhotoLoss = 0.0
        fakePhotoLoss = 0.0
        # photo to monet generator losses
        gen_photo_to_monet_loss = 0.0
        cyc_loss_fake_photo = 0.0
        identity_fake_photo_loss = 0.0
        # monet to photo generator losses
        gen_monet_to_photo_loss = 0.0
        cyc_loss_fake_monet = 0.0
        identity_fake_monet_loss = 0.0
        
        monet_dis.train() 
        photo_dis.train() 

        monet_to_photo.eval()
        photo_to_monet.eval()
        
        do_epochs = discriminator_epochs if epoch % 100 != 0 else discriminator_epochs * 10

        for _ in progress_bar(range(do_epochs), parent=mb):

            # train monet discriminator: monet_dis (#photo->monet)
            monet_iter = iter(monet_loader)
            photo_iter = iter(photo_loader)

            for monet_batch in monet_iter:
                monet_batch = monet_batch.to(device)

                opt_mdis.zero_grad()
                
                real_monet = monet_dis(monet_batch)
                fake_monet = photo_to_monet(next(photo_iter).to(device))
                
                # compute the monet discriminator losses on real images
                realMonetLoss = real_mse_loss(real_monet, adverserial_weight)
                # compute the monet discriminator losses on fake images
                fakeMonetLoss = fake_mse_loss(fake_monet, adverserial_weight)
                total_monet_dis_loss = realMonetLoss + fakeMonetLoss
                
                # do back propagation
                total_monet_dis_loss.backward()
                opt_mdis.step()

            # train photo discriminator (#monet->photo)
            monet_iter = iter(monet_loader)
            photo_iter = iter(photo_loader)

            for monet_batch in monet_iter:
                monet_batch = monet_batch.to(device)
                photo_batch = next(photo_iter).to(device)

                opt_pdis.zero_grad()
                
                real_photo = photo_dis(photo_batch)
                fake_photo = monet_to_photo(monet_batch)
                
                # compute the photo discriminator losses on real images
                realPhotoLoss = real_mse_loss(real_photo, adverserial_weight)
                # compute the photo discriminator losses on fake images
                fakePhotoLoss = fake_mse_loss(fake_photo, adverserial_weight)
                total_photo_dis_loss = realPhotoLoss + fakePhotoLoss
                
                # do back propagation
                total_photo_dis_loss.backward()
                opt_pdis.step()

        monet_iter = iter(monet_loader)
        photo_iter = iter(photo_loader)

        monet_to_photo.train()
        photo_to_monet.train()

        monet_dis.eval()
        photo_dis.eval()

        # train photo_to_monet generator (photo -> monet and generated monet -> reconstructed photo)
        for _ in progress_bar(range(len(monet_iter)), parent=mb):
            photo_batch = next(photo_iter).to(device)

            opt_p2m.zero_grad()
            opt_m2p.zero_grad()

            # convert the photo image to monet style
            fake_monet = photo_to_monet(photo_batch)
            dis_fake_monet = monet_dis(fake_monet)
            # compute generator loss based on monet domain
            gen_photo_to_monet_loss = real_mse_loss(dis_fake_monet, adverserial_weight)
            fake_photo = monet_to_photo(fake_monet)
            # compute cycle consistency loss
            cyc_loss_fake_photo = cycle_consistency_loss(photo_batch, fake_photo, lambda_weight)
            # compute identity loss
            identity_fake_photo_loss = identity_loss(photo_batch, fake_monet, identity_weight)
            
            total_gen_photo_to_monet_loss = gen_photo_to_monet_loss + cyc_loss_fake_photo + identity_fake_photo_loss
            
            # do back propagation on the total generator loss
            total_gen_photo_to_monet_loss.backward()

            opt_p2m.step()
            opt_m2p.step()

        # train monet_to_photo generator (monet -> photot and generated photo -> reconstructed monet)
        for monet_batch in progress_bar(monet_iter, parent=mb):
            monet_batch = monet_batch.to(device)

            opt_p2m.zero_grad()
            opt_m2p.zero_grad()

            # convert the monet image to photo style
            fake_photo = monet_to_photo(monet_batch)
            dis_fake_photo = photo_dis(fake_photo)
            # compute generator loss based on photo domain
            gen_monet_to_photo_loss = real_mse_loss(dis_fake_photo, adverserial_weight)
            fake_monet = photo_to_monet(fake_photo)
            # compute cycle consistency loss
            cyc_loss_fake_monet = cycle_consistency_loss(monet_batch, fake_monet, lambda_weight)
            # compute identity loss
            identity_fake_monet_loss = identity_loss(monet_batch, fake_photo, identity_weight)
            
            total_gen_monet_to_photo_loss = gen_monet_to_photo_loss + cyc_loss_fake_monet + identity_fake_monet_loss

            # do back propagation on the total generator losses
            total_gen_monet_to_photo_loss.backward()

            opt_p2m.step()
            opt_m2p.step() 
            
        # calculate average discriminator and generator losses
        d_total_loss = realMonetLoss + fakeMonetLoss + realPhotoLoss + fakePhotoLoss
        d_total_loss_avg = d_total_loss_avg + d_total_loss/img_per_iteration
        g_total_loss = gen_photo_to_monet_loss + cyc_loss_fake_photo + identity_fake_photo_loss + gen_monet_to_photo_loss + cyc_loss_fake_monet + identity_fake_monet_loss
        g_total_loss_avg = g_total_loss_avg + g_total_loss/img_per_iteration
        
        # append discriminator losses and generator losses to the losses
        losses.append((d_total_loss_avg.item(), g_total_loss_avg.item()))
    
    return losses

In [ ]:
losses = ModelTraining(700)

In [ ]:
# Create a test dataloader and plot the monet and photo images
test_output_batch_size = 1
output_photo_dataloader = DataLoader(photoDS, test_output_batch_size, shuffle=True, num_workers=0, drop_last=True)
_, ax = plt.subplots(3, 2, figsize=(8, 8))

for i in range(3):
    img_iter = iter(output_photo_dataloader)
    img = img_iter.next()
    grid_normalized_photo = make_grid(img, nrow=4)
    grid_original_photo = unnormalize_img(grid_normalized_photo)
    
    generated_monet = photo_to_monet(img.to(device)).cpu().detach()
    grid_normalized_monet = make_grid(generated_monet, nrow=4)
    grid_original_monet = unnormalize_img(grid_normalized_monet)
    
    ax[i, 0].imshow(grid_original_photo.permute(1, 2, 0).detach())
    ax[i, 1].imshow(grid_original_monet.permute(1, 2, 0))
    ax[i, 0].set_title("Photo")
    ax[i, 1].set_title("Monet")
    ax[i, 0].axis("off")
    ax[i, 1].axis("off")

plt.show()

In [ ]:
#Plot loss functions over training
fig, ax = plt.subplots(figsize=(12,8))
losses = np.array(losses)
plt.plot(losses.T[0], label='Discriminators', alpha=0.5)
plt.plot(losses.T[1], label='Generators', alpha=0.5)
plt.title("Training Losses")
plt.legend()
plt.show()

## Submission
To submit our result, we ran our photos through out photo_to_monet generator, did some transformations and then saved them in the images folder.
Since we chaged the method of opening a image in the PictureDataset, we had to chagen the submission file as well.

In [ ]:
final_batch_size = 100
photo_final_ds = PictureDataset(df_photo, path/'photo_jpg')
photo_final_loader = DataLoader(photo_final_ds, batch_size=final_batch_size, drop_last=True)

photo_to_monet.eval()


path_output = Path('./')
!mkdir ../images
for i, photo_batch in enumerate(iter(photo_final_loader)):
    monet_batch = photo_to_monet(photo_batch.to(device))
    monet_batch = monet_batch.detach().cpu()
    for j in range(final_batch_size):
        monet = monet_batch[j]
        monet = unnormalize_img(monet).numpy().astype(np.uint8)
        monet = np.transpose(monet, [1, 2, 0])
        monet = Image.fromarray(monet)
        monet.save('../images/' +  str(i*final_batch_size + j) + '.jpg')

In [ ]:
import shutil
shutil.make_archive("/kaggle/working/images", 'zip', "/kaggle/images")